<a href="https://colab.research.google.com/github/joms-hub/SEE/blob/main/output/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Sessa Empirical Estimator (SEE)

*Assignment by Jomar Cunado and January Venice Toledo*

---
## Background and Purpose  
The **Sessa Empirical Estimator (SEE)** is a data-driven method designed to estimate the duration of prescription drug use, particularly when information on dosage and daily consumption is incomplete or unavailable. It offers a more **systematic and objective** approach compared to traditional methods, which often rely on assumptions.

In pharmacoepidemiology—the study of medication use and its effects on populations—it is crucial to determine how long patients remain on a given treatment. Understanding this duration helps researchers assess **treatment adherence, effectiveness, and potential risks**. However, prescription records often lack details on intended treatment duration, making it difficult to track continuous drug exposure accurately.

To address this, SEE was developed as a **more precise alternative** to traditional approaches like **Researcher-Defined Duration (RDD)**, which assumes a fixed daily dosage. Instead, SEE uses **real-world prescription patterns** to estimate treatment duration, minimizing bias and improving accuracy.

## How It Works  
SEE operates in several key steps:

1. **Analyzing prescription refill patterns**  
   - The method examines how often patients refill their prescriptions. To avoid distortions caused by irregular refill behaviors, SEE filters out unusually long gaps between refills.

2. **Selecting representative prescription pairs**  
   - Instead of analyzing every refill, SEE selects one random pair per patient to **avoid overrepresentation** of frequent refills.

3. **Grouping similar refill behaviors**  
   - Using a **k-means clustering algorithm**, SEE identifies different prescription duration patterns across the patient population. This ensures that prescription lengths are estimated based on **realistic usage trends** rather than arbitrary assumptions.

4. **Determining treatment exposure periods**  
   - For each cluster, SEE calculates a **median prescription duration** and applies it to estimate how long a patient was likely exposed to the medication.

## Effectiveness and Accuracy  
SEE has been tested in both **simulated data** and **real-world prescription records**:

- In controlled simulations, it achieved **96% accuracy** in estimating prescription durations.
- In studies using Danish healthcare data, SEE demonstrated **high sensitivity (78–95%)**, outperforming traditional methods.
- When compared to RDD, SEE showed **lower error rates and greater adaptability** across different drug types.

## Applications in Research  
SEE has been applied in several studies to improve our understanding of **medication adherence and co-exposure** to multiple drugs. Examples include:

- **Antihypertensive medications**: SEE was used to track how long patients remained on combination therapies.
- **Drug safety studies**: It helped assess whether prolonged exposure to the anti-seizure drug **lamotrigine** was linked to an increased risk of cardiovascular events.
- **Pharmacoepidemiological research**: SEE has been integrated into frameworks to enhance the accuracy of drug exposure assessments.

## Significance
By relying on **data-driven insights rather than fixed assumptions**, SEE provides a **more reliable and flexible** way to estimate prescription durations. This improves the quality of research in drug safety, treatment effectiveness, and public health policy, ultimately helping to ensure that medications are used safely and effectively in real-world settings.


# A. INITIALIZATION

## I. Import Libraries



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# DBScan Libraries
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# Check if file exists before attempting to load dataset
from pathlib import Path

## II. Setup Sample Data


In [ ]:
file_path = './csv/med_events.csv'

if Path(file_path).is_file():
    medical_events = pnd.read_csv(file_path)
    print("CSV file loaded successfully!")

    # Convert date column to datetime (assuming 'DATE' is the column name in your CSV)
    medical_events["DATE"] = pnd.to_datetime(medical_events["DATE"])

    # Change column names
    medical_events.columns = ["pnr", "eksd", "perday", "ATC", "dur_original"]

    # Display the first few rows
    print(medical_events.head())
else:
    print(f"Error: File not found at {file_path}. Ensure the file exists.")

# B. Callable Functions

## I. SEE using **K-means**

In [ ]:
def see_k(arg1):
    # Filter dataset for the specified medication
    filtered_data = medical_events[medical_events["ATC"] == arg1].copy()

    # Sort and create a prior prescription date column
    filtered_data = filtered_data.sort_values(["pnr", "eksd"]).copy()
    filtered_data["prev_eksd"] = filtered_data.groupby("pnr")["eksd"].shift(1)

    # Remove rows where prev_eksd is NaN
    filtered_data = filtered_data.dropna().copy()

    # Select one random record per patient
    filtered_data = filtered_data.groupby("pnr").sample(n=1, random_state=5678).copy()

    # Compute event interval (time difference)
    filtered_data["event_interval"] = (filtered_data["eksd"] - filtered_data["prev_eksd"]).dt.days

    # Remove non-positive event intervals
    filtered_data = filtered_data[filtered_data["event_interval"] > 0].copy()

    # Compute ECDF
    if filtered_data.empty:
        return pnd.DataFrame()  # Return empty DataFrame if no valid data

    sorted_intervals = nmp.sort(filtered_data["event_interval"])
    ecdf_y = nmp.arange(1, len(sorted_intervals) + 1) / len(sorted_intervals)

    # Keep only the lower 80% of ECDF
    max_value = sorted_intervals[int(0.8 * len(sorted_intervals))]
    filtered_subset = filtered_data[filtered_data["event_interval"] <= max_value].copy()

    # Ensure filtered_subset is not empty
    if filtered_subset.empty:
        return pnd.DataFrame()  # Return empty DataFrame if no valid data

    # Remove non-positive values before applying log
    filtered_subset = filtered_subset[filtered_subset["event_interval"] > 0]

    # Density plot of log-transformed event intervals
    if len(filtered_subset) > 1:  # Ensure there's enough data for KDE
        kde = gaussian_kde(nmp.log(filtered_subset["event_interval"]))
        x_values = nmp.linspace(min(nmp.log(filtered_subset["event_interval"])),
                                max(nmp.log(filtered_subset["event_interval"])), 100)
        y_values = kde(x_values)
    else:
        return pnd.DataFrame()  # Return empty DataFrame if insufficient data for KDE

    # Silhouette Analysis
    scaler = StandardScaler()
    scaled_intervals = scaler.fit_transform(x_values.reshape(-1, 1))

    best_clusters = []
    best_score = -1
    optimal_k = 2

    for k in range(2, 10):
        kmeans = KMeans(n_clusters=k, random_state=5678, n_init=10)
        labels = kmeans.fit_predict(scaled_intervals)
        score = silhouette_score(scaled_intervals, labels)
        best_clusters.append((k, score))
        if score > best_score:
            best_score = score
            optimal_k = k

    # K-means Clustering
    filtered_subset = filtered_subset.copy()  # Ensure a fresh copy before modification
    filtered_subset.loc[:, "cluster"] = KMeans(n_clusters=optimal_k, random_state=5678, n_init=10).fit_predict(filtered_subset[["event_interval"]])

    # Cluster summary
    cluster_summary = filtered_subset.groupby("cluster")["event_interval"].agg(["min", "max", "median"]).reset_index()

    # Merge clusters back to main dataset
    results = filtered_subset.merge(cluster_summary, on="cluster", how="left")
    results = results[(results["event_interval"] >= results["min"]) & (results["event_interval"] <= results["max"])]

    # Assign cluster to all data
    final_result = medical_events[medical_events["ATC"] == arg1].copy()
    final_result = final_result.merge(results[["pnr", "median", "cluster"]], on="pnr", how="left")

    # Fill missing cluster values safely
    final_result["cluster"] = final_result["cluster"].fillna(0)
    final_result["median"] = final_result["median"].fillna(results["median"].median())

    return final_result

## II. SEE using **DBScan**

In [ ]:
def see_dbscan(arg1):
    # Filter dataset for the specified medication
    filtered_data = medical_events[medical_events["ATC"] == arg1].copy()

    # Sort and create a prior prescription date column
    filtered_data = filtered_data.sort_values(["pnr", "eksd"]).copy()
    filtered_data["prev_eksd"] = filtered_data.groupby("pnr")["eksd"].shift(1)

    # Remove rows where prev_eksd is NaN
    filtered_data = filtered_data.dropna().copy()

    # Select one random record per patient
    filtered_data = filtered_data.groupby("pnr").sample(n=1, random_state=5678).copy()

    # Compute event interval (time difference)
    filtered_data["event_interval"] = (filtered_data["eksd"] - filtered_data["prev_eksd"]).dt.days

    # Remove non-positive event intervals
    filtered_data = filtered_data[filtered_data["event_interval"] > 0].copy()

    # Compute ECDF
    if filtered_data.empty:
        return pnd.DataFrame()  # Return empty DataFrame if no valid data

    sorted_intervals = nmp.sort(filtered_data["event_interval"])
    ecdf_y = nmp.arange(1, len(sorted_intervals) + 1) / len(sorted_intervals)

    # Keep only the lower 80% of ECDF
    max_value = sorted_intervals[int(0.8 * len(sorted_intervals))]
    filtered_subset = filtered_data[filtered_data["event_interval"] <= max_value].copy()

    # Ensure filtered_subset is not empty
    if filtered_subset.empty:
        return pnd.DataFrame()  # Return empty DataFrame if no valid data

    # Remove non-positive values before applying log
    filtered_subset = filtered_subset[filtered_subset["event_interval"] > 0]

    # Density plot of log-transformed event intervals
    if len(filtered_subset) > 1:  # Ensure there's enough data for KDE
        kde = gaussian_kde(nmp.log(filtered_subset["event_interval"]))
        x_values = nmp.linspace(min(nmp.log(filtered_subset["event_interval"])),
                                max(nmp.log(filtered_subset["event_interval"])), 100)
        y_values = kde(x_values)
    else:
        return pnd.DataFrame()  # Return empty DataFrame if insufficient data for KDE

    # Normalize the data for clustering
    scaler = StandardScaler()
    scaled_intervals = scaler.fit_transform(filtered_subset[["event_interval"]])

    # Determine the optimal epsilon (eps) using k-distance method
    if len(scaled_intervals) < 5:
        return pnd.DataFrame()  # Return empty DataFrame if not enough data for DBSCAN

    neighbor = NearestNeighbors(n_neighbors=5)
    neighbor.fit(scaled_intervals)
    distances, _ = neighbor.kneighbors(scaled_intervals)
    k_distances = nmp.sort(distances[:, -1])  # Take the 5th neighbor distance

    # Choose eps as the "knee" point (approximation: 90th percentile of distances)
    eps_value = nmp.percentile(k_distances, 90)

    # Apply DBSCAN clustering
    dbscan = DBSCAN(eps=eps_value, min_samples=5)
    filtered_subset.loc[:, "cluster"] = dbscan.fit_predict(scaled_intervals)

    # Handle noise points (DBSCAN assigns -1 to noise)
    filtered_subset.loc[filtered_subset["cluster"] == -1, "cluster"] = nmp.nan

    # Cluster summary
    cluster_summary = filtered_subset.groupby("cluster")["event_interval"].agg(["min", "max", "median"]).reset_index()

    # Merge clusters back to main dataset
    results = filtered_subset.merge(cluster_summary, on="cluster", how="left")
    results = results[(results["event_interval"] >= results["min"]) & (results["event_interval"] <= results["max"])]

    # Assign cluster to all data
    final_result = medical_events[medical_events["ATC"] == arg1].copy()
    final_result = final_result.merge(results[["pnr", "median", "cluster"]], on="pnr", how="left")

    # Fill missing cluster values safely
    final_result["cluster"] = final_result["cluster"].fillna(0)
    final_result["median"] = final_result["median"].fillna(results["median"].median())

    return final_result

# C. Execution

## I. Using **K-means**


In [ ]:
med_a = see("medA")
med_a